## Transform Irregular / Variable Length Data Stored in a Single Column Into Long Format Suitable for Analysis

This script was written in response to the following question from a DataQuest member:

https://community.dataquest.io/t/how-to-split-a-table-column-which-contains-a-list-and-put-it-in-a-separate-normalised-table/549513

Given a table that initially looks like the output of cell 2, we can use pandas to transform the data into the output of [cell 11](#Long-Format):

In [12]:
import pandas as pd

In [13]:
dictionary = {'VendorID': [6, 7, 8, 9],
              'VendorTagID': ['1,2',
                              '2,3,4',
                              '1',
                              '1,2,3,4']
             }
given_df = pd.DataFrame.from_dict(dictionary)
given_df

,VendorID,VendorTagID
0,6,"1,2"
1,7,"2,3,4"
2,8,1
3,9,"1,2,3,4"


### Assuming that the separator for all data points in column 'VendorTagID' is a single comma, calculate the number of columns needed to be able to transform to a wide format (multiple columns)

In [14]:
tag_qty = given_df['VendorTagID'].apply(lambda l: len(l.split(',')))
max_cols_needed = max(tag_qty)
print(max_cols_needed)

4


### Nested loop that iterates through rows first and individual tags in column 'VendorTagID' second, then creates and populates new columns with all tags separately

In [15]:
for index in given_df.index:
    tags = given_df.loc[index, 'VendorTagID'].split(',')
    for col, tag in enumerate(tags):
        given_df.loc[index, 'Tag ' + str(col)] = tag

### The cell below shows the wide format with the original 'VendorTagID' intact

In [16]:
given_df

,VendorID,VendorTagID,Tag 0,Tag 1,Tag 2,Tag 3
0,6,"1,2",1,2,NaN,NaN
1,7,"2,3,4",2,3,4,NaN
2,8,1,1,NaN,NaN,NaN
3,9,"1,2,3,4",1,2,3,4


### Remove the original 'VendorTagID' column

In [17]:
output_df = given_df.drop(columns='VendorTagID')

### Apply dataframe method '.melt'

#### Prior to running '.melt'

In [18]:
output_df

,VendorID,Tag 0,Tag 1,Tag 2,Tag 3
0,6,1,2,NaN,NaN
1,7,2,3,4,NaN
2,8,1,NaN,NaN,NaN
3,9,1,2,3,4


#### Apply '.melt'

In [19]:
output_df = output_df.melt(id_vars='VendorID')

#### After running '.melt'

In [20]:
output_df

,VendorID,variable,value
0,6,Tag 0,1
1,7,Tag 0,2
2,8,Tag 0,1
3,9,Tag 0,1
4,6,Tag 1,2
5,7,Tag 1,3
6,8,Tag 1,NaN
7,9,Tag 1,2
8,6,Tag 2,NaN
9,7,Tag 2,4


### Final adjustments

At this point, we need to:

* remove the `variable` column containing the original temporary column headers
* remove rows where `value` is NaN
* rename the `value` column to the original `VendorTagID'
* sort

In [21]:
output_df.drop(columns='variable', inplace=True)
output_df.rename(columns={'value': 'VendorTagID'}, inplace=True)
output_df.dropna(inplace=True)
output_df.sort_values('VendorID', inplace=True)

### Long Format

In [22]:
output_df

,VendorID,VendorTagID
0,6,1
4,6,2
1,7,2
5,7,3
9,7,4
2,8,1
3,9,1
7,9,2
11,9,3
15,9,4
